In [14]:
import numpy as np
import six
from itertools import product

In [3]:
DATA = """
.#.
..#
###
""".strip()

In [111]:
class Solver:
    
    def __init__(self, layer):
        self.layout = {}
        for y, row in enumerate(layer.splitlines()):
            for x, value in enumerate(row):
                self.layout[(x, y, 0)] = value == '#'
        
    def __str__(self):
        s = ""
        for z in self.depths:
            s += ' Depth: ' + str(z) + '\n'
            for y in self.heights:
                for x in self.widths:
                    s += '#' if self.read(x, y, z) else '.'
                s += '\n'
            s += '\n\n'
        return s
                
        
    @property
    def depths(self):
        return sorted(set(z for _, _, z in self.layout))
    
    @property
    def widths(self):
        return sorted(set(x for x, _, _ in self.layout))
    
    @property
    def heights(self):
        return sorted(set(y for _, y, _ in self.layout))
                
    def read(self, x, y, z):
        return self.layout.get((x, y, z), False)
    
    def flip(self, x, y, z):
        self.layout[(x, y, z)] = not self.layout[(x, y, z)]
                
    def neighbours(self, x, y, z):
        for i, (dx, dy, dz) in enumerate(product((-1, 0, +1), (-1, 0, +1), (-1, 0, +1))):
            if (dx, dy, dz) == (0, 0, 0): continue
            nx, ny, nz = x + dx, y + dy, z + dz
            yield (nx, ny, nz), self.read(nx, ny, nz)
    
    def step(self):
        
        toCreate = set()
        for (x, y, z), v in six.iteritems(self.layout):
            if not v: continue
            neighbours = list(self.neighbours(x, y, z))
            for (nx, ny, nz), _ in neighbours:
                if (nx, ny, nz) not in self.layout:
                    toCreate.add((nx, ny, nz))
        for (x, y, z) in toCreate:
            self.layout[(x, y, z)] = False
            
        toFlip = set()
        
        for (x, y, z), v in six.iteritems(self.layout):
            neighbours = list(self.neighbours(x, y, z))
            numberOfActiveNeighbours = sum(isActive for _, isActive in neighbours)
            if v and numberOfActiveNeighbours not in (2, 3):
                toFlip.add((x, y, z))
            elif not v and numberOfActiveNeighbours == 3:
                toFlip.add((x, y, z))
        print(toFlip)
        for (x, y, z) in toFlip:
            self.flip(x, y, z)
            
    def count(self):
        return sum(six.itervalues(self.layout))

In [112]:
z = Solver(DATA)
print(z)
z.step()
print(z)

 Depth: 0
.#.
..#
###



{(0, 1, 0), (0, 2, 0), (1, 3, 1), (1, 3, -1), (1, 0, 0), (2, 2, -1), (2, 2, 1), (1, 3, 0), (0, 1, -1), (0, 1, 1)}
 Depth: -1
.....
.....
.#...
...#.
..#..


 Depth: 0
.....
.....
.#.#.
..##.
..#..


 Depth: 1
.....
.....
.#...
...#.
..#..



